In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

from  bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
import time

import warnings
warnings.filterwarnings('ignore')

import string, nltk, re
from konlpy.tag import Okt
from wordcloud import WordCloud

from PIL import Image
from wordcloud import ImageColorGenerator

import os
driver_name =  os.environ['HOMEDRIVE'] + os.environ['HOMEPATH'] + '\\Downloads\\chromedriver_win32\\chromedriver.exe'

### 영화관입장권통합전산망 KOBIS 에서 영화 정보 크롤링

In [2]:
url = "https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do"

home_url = "https://www.kobis.or.kr/"

In [4]:
driver = webdriver.Chrome(driver_name)
driver.get(url)
time.sleep(2)

In [4]:
lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
print(len(lis))

10


In [6]:
driver = webdriver.Chrome(driver_name)
driver.get(url)
time.sleep(2)

TOTAL_COUNT = 1000
START_COUNT = 0
SUB_COUNT = 0
for i in range(START_COUNT):
    driver.find_element(By.CSS_SELECTOR, '.paging > a.btn.next').click()
    time.sleep(3)
    
lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
current = 0
for k in range(SUB_COUNT):
    current += 1
    lis[current].click()
    time.sleep(3)
    lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')

lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')

In [5]:
TOTAL_COUNT = 1000

In [6]:

# driver = webdriver.Chrome(driver_name)

# driver.get(url)
# time.sleep(2)
ORDER = 'movie하'
n_page = 0
# paging
lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
# print(lis)

end, current = len(lis), 0
lines = []

while True:

    try:
        trs = driver.find_elements(By.CSS_SELECTOR, 'tbody > tr')
    except:
        print('nPgae = ', n_page)
        break

    for tr in trs:

        td = tr.find_elements(By.TAG_NAME, 'td')

        kind = td[5].text
        genre = td[6].text
        
        if kind =='' or kind =='단편' or kind == "기타" or kind == '옴니버스':
            continue
        if '성인' in genre or genre == '' :
            continue

        td[0].find_element(By.TAG_NAME, 'a').click()
        time.sleep(3)

        # 영화 제목
        title = driver.find_element(By.CSS_SELECTOR, '.hd_layer > div').text
        find_index = title.find(')') 
        if find_index >= 0 :
            title = title[:find_index+1]
        
        infos = driver.find_element(By.CSS_SELECTOR, 'div.basic')

        # 시놉시스가 없으면 pass
        try :
            synopsis = infos.find_element(By.CSS_SELECTOR, '.info.info2 > p').text
        except :
            driver.find_element(By.CSS_SELECTOR, '.hd_layer > a.close').click()
            time.sleep(3)
            continue

        infos_head = infos.find_element(By.CSS_SELECTOR, '.ovf.info.info1')
            
        # 이미지 URL
        img = infos_head.find_element(By.CSS_SELECTOR, 'a > img').get_attribute('src')

        # 요약 정보
        dds = infos_head.find_elements(By.CSS_SELECTOR, 'dl > dd')

        # 코드 정보
        code = dds[0].text
        summering = dds[3].text
        if (summering.find('멜로/로맨스') >=0 or summering.find('드라마') >=0 ) and summering.find('청소년관람불가') >= 0:
            driver.find_element(By.CSS_SELECTOR, '.hd_layer > a.close').click()
            time.sleep(3)
            continue

        # 개봉일
        first_day = dds[-6].text
        # 제작년도
        production_year = dds[-5].text
            
        # 감독 / 주연 배우
        tmp = infos.find_elements(By.CSS_SELECTOR, '.info.info2 > .staffMore > dl > div')


        movie_director, star_actor = '', ''
        find_count = 0

        # 감독 : 있는 경우 : 첫 번째에 감독이 없으면 감독이 없는 데이터다
        if len(tmp) > 0 :
            nindex = tmp[0].text.find('감독')
            if nindex >= 0:
                movie_director = tmp[0].text[nindex + 2:].strip()
                find_count += 1

        # 주연 배우
        for tp in tmp[find_count:]:
            
            text = tp.text.split('\n')            
            if len(text) < 2 : continue

            nfind = text[1].find('[주연]')
            
            if nfind >= 0:
                star_actor = text[1][nfind + 4: ].strip()
                # print(star_actor)
                break

        # print(title, ' - ', movie_director, ' - ', star_actor)
        lines.append({'code': code, 'title': title, 'genre': genre, 'summering': summering, 
                    'first_day': first_day, 'production_year': production_year, 
                    'movie_director':movie_director, 'star_actor': star_actor, 
                    'img': img, 'synopsis': synopsis})

        driver.find_element(By.CSS_SELECTOR, '.hd_layer > a.close').click()
        time.sleep(3)


    current += 1
    n_page += 1

    # if n_page == TOTAL_COUNT: break

    if end == current:
        try:
            driver.find_element(By.CSS_SELECTOR, '.paging > a.btn.next').click()
            time.sleep(3)
            lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
            end, current = len(lis), 0
        except:
            print('except2 nPgae = ', n_page)
            break
    else:
        # 다음 페이지 넘어가기
        lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
        # print(lis)
        lis[current].click()
        time.sleep(3)

df = pd.DataFrame(lines)

df.to_csv(f'../static/data/{ORDER}.csv', index=False)

except2 nPgae =  99


In [6]:
len(lines)

565

In [10]:
df.head(5)

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis
0,20231889,보 이즈 어프레이드 (Beau Is Afraid),어드벤처(1),"장편 | 일반영화 | 어드벤처, 공포(호러) | 178분 58초 | 청소년관람불가 ...",2023-07-05,2023년,아리 에스터,호아킨 피닉스,https://www.kobis.or.kr/common/mast/movie/2023...,편집증을 앓는 ‘보’와 그를 집착적으로 사랑하는 엄마 ‘모나’\n\n엄마를 무조건 ...
1,2023C925,빌리 보이 (Juvenile),범죄(2),"온라인전용 | 일반영화 | 범죄, 드라마, 스릴러 | 86분 08초 | 미국",해당정보없음,2017년,,,https://www.kobis.or.kr/common/mast/movie/2023...,가난하고 비참한 환경에서 자라온 빌리는 절친 마이키에게 도둑질과 싸움을 배웠다. 둘...
2,2023C942,블러드 카 (Blood Car),코미디(1),"온라인전용 | 일반영화 | 코미디, 공포(호러) | 75분 49초 | 미국",해당정보없음,2007년,,,https://www.kobis.or.kr/kobis/web/comm/images/...,기름값이 천정부지로 치솟는 가까운 미래에 채식주의자 선생님 아치 앤드류스는 밀싹 연...
3,20227552,비닐하우스 (Greenhouse),범죄,장편 | 일반영화 | 범죄 | 100분 29초 | 15세이상관람가 | 한국,2023-07-26,2022년,,김서형,https://www.kobis.or.kr/common/mast/movie/2023...,집이 없어 비닐하우스에 기거하지만 하루하루 성실하게 살아가고 있던 요양보호사 문정이...
4,20231828,보통의 카스미 (I am What I am),드라마,장편 | 예술영화 | 드라마 | 104분 27초 | 12세이상관람가 | 일본,2023-07-19,2022년,,미우라 토코,https://www.kobis.or.kr/common/mast/movie/2023...,“난 연애도 안 하고 싶고 애초에 그런 감정도 없고 혼자서 살 수 있고\n그게 쓸쓸...


In [67]:
df.shape

(10, 10)

In [7]:
df = pd.DataFrame(lines)

In [7]:
driver.close()

In [8]:
df.tail()

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis
560,20149678,뱀파이어에 관한 아주 특별한 다큐멘터리 (What We Do in the Shadows),코미디,"장편 | 일반영화 | 코미디 | 86분 15초 | 뉴질랜드, 미국",해당정보없음,2014년,제메인 클레멘트,,https://www.kobis.or.kr/kobis/web/comm/images/...,"뉴질랜드에서 매년 개최되는 가면무도회에는 좀비, 마술사, 뱀파이어, 늑대인간 등 초..."
561,20207415,빛의 아이 (Daughter of the Light),다큐멘터리,장편 | 일반영화 | 다큐멘터리 | 100분 0초 | 중국,해당정보없음,2019년,,,https://www.kobis.or.kr/kobis/web/comm/images/...,"13살의 메톡 카르포는 티벳의 고아 기숙학교에 머무르고 있지만, 그녀의 부모는 이혼..."
562,20207388,블랙 독 (Black Dog),액션(1),"장편 | 일반영화 | 액션, 스릴러 | 99분 57초 | 15세이상관람가 | 기타",해당정보없음,2019년,,,https://www.kobis.or.kr/common/mast/movie/2020...,안드레이는 기억을 잃고 시리아에서 러시아로 투항하다. 자신의 이름도 직업도 기억해내...
563,19930026,부활의 노래 (The Song Of Resurrection),드라마,장편 | 독립영화 | 드라마 | 92분 0초 | 중학생이상관람가 | 한국,1991-03-01,1990년,이정국,김해연 | 박지수 | 이경영,https://www.kobis.or.kr/kobis/web/comm/images/...,"사회정의실현을 꿈꾸던 철기는 야학에 참여하면서 정치,사회적 모순과 민중의 현실에 대..."
564,20207044,빌마르크 어사일럼 (Villmark 2),공포(호러),장편 | 일반영화 | 공포(호러) | 93분 47초 | 기타,해당정보없음,2015년,,안데르스 바스모 크리스티안센 | 엘렌 도리트 페테르센,https://www.kobis.or.kr/kobis/web/comm/images/...,"지옥보다 더한 그곳\n광기만이 존재하는 ‘빌마르크 어사일럼＇\n\n깊은 산골짜기, ..."


In [9]:
df.to_csv(f'../static/data/{ORDER}.csv', index=False)